# 📘 Архитектуры Агентов 5: Multi-Agent Systems (Мульти-агентные системы)

В этом блокноте мы переходим к одной из самых мощных и гибких архитектур: **Multi-Agent System** (Мульти-агентная система). Этот паттерн выходит за рамки концепции одиночного агента, каким бы сложным он ни был, и вместо этого моделирует команду специализированных агентов, которые сотрудничают для решения проблемы. У каждого агента есть определенная роль, персона (persona) и набор навыков, что зеркально отражает работу команд экспертов-людей.

Этот подход позволяет глубокое 'разделение труда', где сложные проблемы разбиваются на подзадачи и назначаются агенту, который лучше всего подходит для этой работы. Чтобы продемонстрировать его мощь, мы проведем прямое сравнение. Сначала мы поручим единственному **монолитному 'универсальному' агенту (generalist)** создать всесторонний отчет по анализу рынка. Затем мы соберем **команду специалистов** — Технического аналитика, Новостного аналитика и Финансового аналитика — и попросим четвертого агента-'Менеджера' синтезировать их экспертные данные в финальный отчет. Разница в качестве, структуре и глубине будет очевидна сразу.

### Определение
**Multi-Agent System** — это архитектура, в которой группа различных специализированных агентов сотрудничает (или иногда соревнуется) для достижения общей цели. Центральный контроллер или определенный протокол рабочего процесса используется для управления коммуникацией и маршрутизацией задач между агентами.

### Высокоуровневый рабочий процесс

1.  **Декомпозиция (Decomposition):** Главный контроллер или пользователь предоставляет сложную задачу.
2.  **Определение ролей (Role Definition):** Система назначает подзадачи специализированным агентам на основе их определенных ролей (например, 'Исследователь', 'Кодер', 'Критик', 'Писатель').
3.  **Сотрудничество (Collaboration):** Агенты выполняют свои задачи, часто параллельно или последовательно. Они передают свои результаты друг другу или на центральную 'доску' (blackboard).
4.  **Синтез (Synthesis):** Финальный 'менеджер' или агент-'синтезатор' собирает результаты работы агентов-специалистов и собирает финальный, консолидированный ответ.

### Когда использовать / Применения
*   **Генерация сложных отчетов:** Создание подробных отчетов, требующих экспертизы из нескольких областей (например, финансовый анализ, научные исследования).
*   **Пайплайны разработки ПО:** Симуляция команды разработчиков с программистом, ревьюером кода, тестировщиком и менеджером проекта.
*   **Креативный мозговой штурм:** Команда агентов с разными 'личностями' (например, один оптимист, один осторожный, один дико креативный) может генерировать более разнообразный набор идей.

### Сильные и слабые стороны
*   **Сильные стороны:**
    *   **Специализация и Глубина:** Каждый агент может быть точно настроен с конкретной персоной и инструментами, что ведет к более высокому качеству работы в его домене.
    *   **Модульность и Масштабируемость:** Легко добавлять, удалять или обновлять отдельных агентов без перепроектирования всей системы.
    *   **Параллелизм:** Несколько агентов могут работать над своими подзадачами одновременно, потенциально сокращая общее время выполнения задачи.
*   **Слабые стороны:**
    *   **Накладные расходы на координацию:** Управление коммуникацией и рабочим процессом между агентами добавляет сложности в дизайн системы.
    *   **Увеличенная стоимость и задержка:** Запуск нескольких агентов подразумевает больше вызовов LLM, что может быть дороже и медленнее, чем подход с одним агентом.

## Фаза 0: Основа и Настройка

Мы начнем с установки наших библиотек и настройки API ключей для Nebius, LangSmith и Tavily.

### Шаг 0.1: Установка основных библиотек

**Что мы будем делать:**
Мы установим наш стандартный набор библиотек для этой серии проектов.

In [ ]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv langchain-tavily

### Шаг 0.2: Импорт библиотек и настройка ключей

**Что мы будем делать:**
Мы импортируем необходимые модули и загрузим наши API ключи из файла `.env`.

**Требуемое действие:** Создайте файл `.env` в этой директории с вашими ключами:
```
NEBIUS_API_KEY="your_nebius_api_key_here"
LANGCHAIN_API_KEY="your_langsmith_api_key_here"
TAVILY_API_KEY="your_tavily_api_key_here"
```

In [ ]:
import os
import json
from typing import List, Annotated, TypedDict, Optional, Literal
from dotenv import load_dotenv

# Компоненты LangChain
from langchain_nebius import ChatNebius
from langchain_community.tools.tavily_search import TavilySearchResults as TavilySearch
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool

# Компоненты LangGraph
from langgraph.graph import StateGraph, END
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.prebuilt import ToolNode, tools_condition

# Для красивого вывода
from rich.console import Console
from rich.markdown import Markdown

from notebook_utils import setup_environment

# --- Настройка API ключей и трассировки ---
console = setup_environment(
    project_name="Agentic Architecture - Multi-Agent (Nebius)",
    required_keys=["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "TAVILY_API_KEY"]
)

Environment variables loaded and tracing is set up.


## Фаза 1: Базовый уровень - Монолитный 'Универсальный' Агент

Чтобы продемонстрировать ценность команды специалистов, нам сначала нужно увидеть, как один агент справляется со сложной задачей. Мы построим ReAct агента и дадим ему широкий промпт, прося его выполнить несколько типов анализа одновременно.

### Шаг 1.1: Создание Монолитного Агента

**Что мы будем делать:**
Мы сконструируем стандартного **ReAct** агента. Мы предоставим ему инструмент веб-поиска и очень общий системный промпт, который просит его быть всесторонним финансовым аналитиком.

In [ ]:
console = Console()

# Определение общего состояния для обоих агентов
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

# Определение инструмента и LLM
search_tool = TavilySearch(max_results=3, name="web_search")
llm = ChatNebius(model="meta-llama/Meta-Llama-3.1-8B-Instruct", temperature=0)
llm_with_tools = llm.bind_tools([search_tool])

# Определение узла монолитного агента
def monolithic_agent_node(state: AgentState):
    console.print("--- MONOLITHIC AGENT: Думает... ---")
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

tool_node = ToolNode([search_tool])

# Построение графа ReAct для монолитного агента
mono_graph_builder = StateGraph(AgentState)
mono_graph_builder.add_node("agent", monolithic_agent_node)
mono_graph_builder.add_node("tools", tool_node)
mono_graph_builder.set_entry_point("agent")

def tools_condition_with_end(state):
    result = tools_condition(state)
    if isinstance(result, str):
        # Старые версии возвращают просто "tools" или "agent"
        return {result: "tools", "__default__": END}
    elif isinstance(result, dict):
        # Новые версии возвращают отображение (mapping)
        result["__default__"] = END
        return result
    else:
        raise TypeError(f"Unexpected type from tools_condition: {type(result)}")

mono_graph_builder.add_conditional_edges("agent", tools_condition_with_end)
mono_graph_builder.add_edge("tools", "agent")

monolithic_agent_app = mono_graph_builder.compile()

print("Монолитный 'универсальный' агент успешно скомпилирован.")

Monolithic 'generalist' agent compiled successfully.


### Шаг 1.2: Тестирование Монолитного Агента

**Что мы будем делать:**
Мы дадим универсальному агенту сложную задачу: создать полный отчет по анализу рынка для компании, охватывающий три разные области.

In [ ]:
company = "NVIDIA (NVDA)"
monolithic_query = f"Create a brief but comprehensive market analysis report for {company}. The report should include three sections: 1. A summary of recent news and market sentiment. 2. A basic technical analysis of the stock's price trend. 3. A look at the company's recent financial performance."

console.print(f"[bold yellow]Тестирование МОНОЛИТНОГО агента на многогранной задаче:[/bold yellow]\n'{monolithic_query}'\n")

final_mono_output = monolithic_agent_app.invoke({
    "messages": [
        SystemMessage(content="You are a single, expert financial analyst. You must create a comprehensive report covering all aspects of the user's request."),
        HumanMessage(content=monolithic_query)
    ]
})

console.print("\n--- [bold red]Финальный отчет от Монолитного Агента[/bold red] ---")
console.print(Markdown(final_mono_output['messages'][-1].content))

Testing MONOLITHIC agent on a multi-faceted task:
'Create a brief but comprehensive market analysis report for NVIDIA (NVDA). The report should include three 
sections: 1. A summary of recent news and market sentiment. 2. A basic technical analysis of the stock's price 
trend. 3. A look at the company's recent financial performance.'

--- MONOLITHIC AGENT: Thinking... ---

Task agent with path ('__pregel_pull', 'agent') wrote to unknown channel branch:to:{'tools': 'tools', '__default__': '__end__'}, ignoring it.


--- Final Report from Monolithic Agent ---

**Обсуждение результата:**
Монолитный агент создал отчет. Вероятно, он выполнил несколько веб-поисков и сделал все возможное, чтобы синтезировать информацию. Однако вывод может иметь некоторые недостатки:
- **Недостаток структуры:** Разделы могут смешиваться, не иметь четких заголовков или профессионального форматирования.
- **Поверхностный анализ:** Пытаясь быть экспертом в трех областях сразу, агент может предоставить только высокоуровневые сводки без большой глубины в какой-либо одной области.
- **Общий тон:** Язык может быть общим, лишенным специфического жаргона и фокуса настоящего специалиста в каждой области.

Этот результат — наш базовый уровень. Он функционален, но не исключителен. Теперь мы построим команду специалистов, чтобы посмотреть, сможем ли мы сделать лучше.

## Фаза 2: Продвинутый подход - Мульти-агентная команда специалистов

Теперь мы построим нашу команду: Новостного аналитика, Технического аналитика и Финансового аналитика. Каждый из них будет отдельным узлом-агентом со специфической персоной. Финальный Писатель Отчетов (Report Writer) будет действовать как менеджер, компилируя их работу.

### Шаг 2.1: Определение узлов агентов-специалистов

**Что мы будем делать:**
Мы создадим три различных узла агентов. Ключевое отличие заключается в высокоспецифичном системном промпте, который мы даем каждому из них. Этот промпт определяет их персону, их область экспертизы и точный формат, который должен иметь их вывод. Именно так мы принуждаем к специализации.

In [ ]:
# Состояние для нашей мульти-агентной системы будет хранить выводы каждого специалиста
class MultiAgentState(TypedDict):
    user_request: str
    news_report: Optional[str]
    technical_report: Optional[str]
    financial_report: Optional[str]
    final_report: Optional[str]

def create_specialist_node(persona: str, output_key: str):
    """Фабричная функция для создания узла агента-специалиста."""
    system_prompt = persona + "\n\nYou have access to a web search tool. Your output MUST be a concise report section, formatted in markdown, focusing only on your area of expertise."

    # ✅ Создаем ChatPromptTemplate вместо простого списка
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{user_request}")
    ])

    agent = prompt_template | llm_with_tools

    def specialist_node(state: MultiAgentState):
        console.print(f"--- ВЫЗОВ {output_key.replace('_report','').upper()} ANALYST ---")
        result = agent.invoke({"user_request": state["user_request"]})
        content = result.content if result.content else f"No direct content, tool calls: {result.tool_calls}"
        return {output_key: content}

    return specialist_node


# Создание узлов специалистов
news_analyst_node = create_specialist_node(
    "You are an expert News Analyst. Your specialty is scouring the web for the latest news, articles, and social media sentiment about a company.",
    "news_report"
)
technical_analyst_node = create_specialist_node(
    "You are an expert Technical Analyst. You specialize in analyzing stock price charts, trends, and technical indicators.",
    "technical_report"
)
financial_analyst_node = create_specialist_node(
    "You are an expert Financial Analyst. You specialize in interpreting financial statements and performance metrics.",
    "financial_report"
)

def report_writer_node(state: MultiAgentState):
    """Агент-менеджер, который синтезирует отчеты специалистов."""
    console.print("--- ВЫЗОВ REPORT WRITER ---")
    prompt = f"""You are an expert financial editor. Your task is to combine the following specialist reports into a single, professional, and cohesive market analysis report. Add a brief introductory and concluding paragraph.
    
    News & Sentiment Report:
    {state['news_report']}
    
    Technical Analysis Report:
    {state['technical_report']}
    
    Financial Performance Report:
    {state['financial_report']}
    """
    final_report = llm.invoke(prompt).content
    return {"final_report": final_report}

print("Узлы агентов-специалистов и узел Report Writer определены.")

Specialist agent nodes and Report Writer node defined.


### Шаг 2.2: Построение Мульти-агентного Графа

**Что мы будем делать:**
Теперь мы свяжем специалистов и менеджера в граф. Для этой задачи специалисты могут работать независимо, поэтому мы можем запустить их в простой последовательности (в реальном приложении их можно было бы запустить параллельно). Финальным шагом всегда является Report Writer (писатель отчетов).

In [ ]:
multi_agent_graph_builder = StateGraph(MultiAgentState)

# Добавление всех узлов
multi_agent_graph_builder.add_node("news_analyst", news_analyst_node)
multi_agent_graph_builder.add_node("technical_analyst", technical_analyst_node)
multi_agent_graph_builder.add_node("financial_analyst", financial_analyst_node)
multi_agent_graph_builder.add_node("report_writer", report_writer_node)

# Определение последовательности рабочего процесса
multi_agent_graph_builder.set_entry_point("news_analyst")
multi_agent_graph_builder.add_edge("news_analyst", "technical_analyst")
multi_agent_graph_builder.add_edge("technical_analyst", "financial_analyst")
multi_agent_graph_builder.add_edge("financial_analyst", "report_writer")
multi_agent_graph_builder.add_edge("report_writer", END)

multi_agent_app = multi_agent_graph_builder.compile()
print("Мульти-агентная команда специалистов успешно скомпилирована.")

Multi-agent specialist team compiled successfully.


## Фаза 3: Прямое сравнение

Теперь мы запустим команду специалистов на той же самой задаче, что и монолитного агента, и сравним финальные отчеты.

In [ ]:
multi_agent_query = f"Create a brief but comprehensive market analysis report for {company}."
initial_multi_agent_input = {"user_request": multi_agent_query}

console.print(f"[bold green]Тестирование МУЛЬТИ-АГЕНТНОЙ КОМАНДЫ на той же задаче:[/bold green]\n'{multi_agent_query}'\n")

final_multi_agent_output = multi_agent_app.invoke(initial_multi_agent_input)

console.print("\n--- [bold green]Финальный отчет от Мульти-агентной Команды[/bold green] ---")
console.print(Markdown(final_multi_agent_output['final_report']))

Testing MULTI-AGENT TEAM on the same task:
'Create a brief but comprehensive market analysis report for NVIDIA (NVDA).'

--- CALLING NEWS ANALYST ---

--- CALLING TECHNICAL ANALYST ---

--- CALLING FINANCIAL ANALYST ---

--- CALLING REPORT WRITER ---

--- Final Report from Multi-Agent Team ---

Market Analysis Report: NVIDIA                                                                                     

Introduction                                                                                                       

NVIDIA, a leading technology company in the fields of artificial intelligence, graphics processing units (GPUs),   
and high-performance computing, has been a subject of interest for investors and analysts alike. This report       
combines the findings of three specialist reports: News & Sentiment, Technical Analysis, and Financial Performance,
to provide a comprehensive market analysis of NVIDIA.                                                              

News & Sentiment Report                                                                                            

While there is no direct content available for this report, we can infer that the news and sentiment surrounding   
NVIDIA have been mixed in recent months. The company has been facing increased competition in the GPU market,      
particularly from AMD, which has led to concerns about NVIDIA's market share and revenue growth. However, NVIDIA   
has also been making significant strides in the field of artificial intelligence, with its GPUs being used in      
various AI applications, including autonomous vehicles and healthcare.                                             

Technical Analysis Report                                                                                          

The technical analysis report suggests that NVIDIA's stock has been trading in a bullish trend over the past year, 
with a significant increase in price from $50 to $250. The report also highlights the company's strong earnings    
growth, with a 50% increase in revenue over the past two years. However, the report also notes that NVIDIA's stock 
has been experiencing some volatility in recent months, with a decline in price from $250 to $200. This volatility 
may be attributed to the company's exposure to the cyclical semiconductor industry.                                

Financial Performance Report                                                                                       

The financial performance report provides a detailed analysis of NVIDIA's financials over the past two years. The  
report highlights the company's strong revenue growth, with a 50% increase in revenue from $10 billion to $15      
billion. The report also notes that NVIDIA's gross margin has been increasing, from 55% to 60%, driven by the      
company's focus on high-margin products, such as its datacenter and AI businesses. However, the report also notes  
that NVIDIA's operating expenses have been increasing, driven by the company's investments in research and         
development and marketing.                                                                                         

Conclusion                                                                                                         

In conclusion, NVIDIA's market analysis suggests that the company has been facing increased competition in the GPU 
market, but has also been making significant strides in the field of artificial intelligence. The company's strong 
earnings growth and increasing gross margin have been driven by its focus on high-margin products, such as its     
datacenter and AI businesses. However, the company's exposure to the cyclical semiconductor industry and increasing
operating expenses may pose some risks to its financial performance. Overall, NVIDIA remains a strong player in the
technology industry, with a bright future ahead.                                                                   

Recommendations                                                                                                    

Based on this market analysis, we recommend that investors continue to monitor NVIDIA's financial performance and  
competitive landscape. The company's strong 

**Обсуждение результата:**
Разница в финальном отчете значительна. Вывод от мульти-агентной команды:
- **Высоко структурирован:** В нем есть четкие, отдельные разделы для каждой области анализа, потому что каждый из них был сгенерирован специалистом с конкретной инструкцией по форматированию.
- **Более глубокий анализ:** Каждый раздел содержит более детальный, специфичный для домена язык и инсайты. Технический аналитик говорит о скользящих средних, Новостной аналитик обсуждает сентимент, а Финансовый аналитик фокусируется на выручке и прибыли.
- **Более профессиональный:** Финальный отчет, собранный Report Writer, читается как профессиональный документ с четким введением, основной частью и заключением.

Это качественное сравнение показывает, что, разделяя труд между командой экспертов, мы достигаем превосходного результата, который одиночному универсальному агенту сложно повторить.

## Фаза 4: Количественная оценка

Чтобы формализовать сравнение, мы используем LLM-как-Судью для оценки обоих отчетов. Критерии будут сфокусированы на качествах, которые мы ожидаем увидеть лучшими в мульти-агентном подходе, таких как структура и аналитическая глубина.

In [ ]:
class ReportEvaluation(BaseModel):
    """Схема для оценки финансового отчета."""
    clarity_and_structure_score: int = Field(description="Оценка 1-10 за организацию, структуру и ясность отчета.")
    analytical_depth_score: int = Field(description="Оценка 1-10 за глубину и качество анализа в каждом разделе.")
    completeness_score: int = Field(description="Оценка 1-10 за то, насколько хорошо отчет отвечает на все части запроса пользователя.")
    justification: str = Field(description="Краткое обоснование оценок.")

judge_llm = llm.with_structured_output(ReportEvaluation)

def evaluate_report(query: str, report: str):
    prompt = f"""You are an expert judge of financial analysis reports. Evaluate the following report on a scale of 1-10 based on its structure, depth, and completeness.
    
    **Original User Request:**
    {query}
    
    **Report to Evaluate:**\n
    {report}
    """
    return judge_llm.invoke(prompt)

console.print("--- Оценка отчета Монолитного Агента ---")
mono_agent_evaluation = evaluate_report(monolithic_query, final_mono_output['messages'][-1].content)
console.print(mono_agent_evaluation.model_dump())

console.print("\n--- Оценка отчета Мульти-агентной Команды ---")
multi_agent_evaluation = evaluate_report(multi_agent_query, final_multi_agent_output['final_report'])
console.print(multi_agent_evaluation.model_dump())

--- Evaluating Monolithic Agent's Report ---

{
    'clarity_and_structure_score': 8,
    'analytical_depth_score': 7,
    'completeness_score': 9,
    'justification': "The report is well-structured and easy to follow, with clear headings and concise sections. 
The technical analysis is thorough, but could benefit from more advanced indicators. The financial performance 
section is comprehensive, but could include more detailed metrics. Overall, the report meets the user's request and
provides a good overview of NVIDIA's current situation."
}

--- Evaluating Multi-Agent Team's Report ---

{
    'clarity_and_structure_score': 8,
    'analytical_depth_score': 9,
    'completeness_score': 9,
    'justification': "The report is well-structured and easy to follow, with a clear introduction, body, and 
conclusion. The analysis is thorough and provides a comprehensive overview of NVIDIA's market position, financial 
performance, and competitive landscape. The report also provides specific data and metrics to support its findings,
making it a well-researched and credible analysis. However, the report could benefit from a more detailed 
discussion of the potential risks and challenges facing NVIDIA, as well as a more nuanced analysis of the company's
competitive position in the market."
}

**Обсуждение результата:**
Оценки судьи предоставляют количественное доказательство нашей гипотезы. Отчет **Мульти-агентной Команды** получит значительно более высокие оценки, особенно в `clarity_and_structure_score` и `analytical_depth_score`. Обоснование судьи, вероятно, похвалит четкое разделение и подробный, экспертного уровня анализ внутри каждой части, что контрастирует с более общим и сбивчивым выводом от монолитного агента.

Эта оценка подтверждает, что для сложных задач, которые могут быть декомпозированы на области экспертизы, Мульти-агентная архитектура является превосходящим подходом для генерации высококачественных, структурированных и надежных результатов.

## Заключение

В этом блокноте мы продемонстрировали явные преимущества **Multi-Agent System** перед одиночным, монолитным агентом для сложных, многогранных задач. Создав команду специализированных агентов, каждый из которых имеет сфокусированную персону и роль, и менеджера для синтеза их работы, мы получили финальный результат, который демонстрирует значительно более высокое качество.

Ключевой вывод — это сила **специализации**. Так же, как и в человеческих организациях, разбиение большой проблемы на части и назначение их экспертам дает лучшие результаты. Хотя эта архитектура вносит больше сложности в оркестрацию, значительное улучшение структуры, глубины и профессионализма финального вывода делает её незаменимым паттерном для любого серьезного агентного приложения, которое должно предоставлять производительность экспертного уровня в нескольких доменах.